In [29]:
import json
import numpy as np
from elasticsearch import Elasticsearch
from pprint import pprint

In [4]:
# Download the full corpus (822mb) and unzip the 25-dimensional vectors.
if not os.path.exists('/tmp/glove.6B.zip'):
    !curl http://nlp.stanford.edu/data/glove.6B.zip -s -L -o /tmp/glove.6B.zip

if not os.path.exists('/tmp/glove.6B.50d.txt'):
    !unzip -p /tmp/glove.6B.zip glove.6B.50d.txt > /tmp/glove.6B.50d.txt
        
!du -hs /tmp/glove*

164M	/tmp/glove.6B.50d.txt
823M	/tmp/glove.6B.zip


In [11]:
# Read the words into a list and the vectors into a list of lists.
words, vecs = [], []

for line in open('/tmp/glove.6B.50d.txt'):
    tokens = line.split(' ')
    words.append(tokens[0])
    vecs.append([float(x) for x in tokens[1:]])
    
len(words), len(vecs)

(400000, (400000, 50))

In [15]:
# Setup the Elasticsearch client. This assumes you're already running Elasticsearch with Elastiknn installed.
# If that's not the case, see the Installation docs for help.
es = Elasticsearch()
es.info()

{'name': 'elasticsearch_master',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'IS6qMv95QYqzlvVPzjG7Qg',
 'version': {'number': '7.6.2',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'ef48eb35cf30adf4db14086e8aabd07ef6fb113f',
  'build_date': '2020-03-26T06:34:37.794943Z',
  'build_snapshot': False,
  'lucene_version': '8.4.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [18]:
# Create the index. Using 1 shard with 0 replicas for simplicity.
index = "elastiknn-example-glove-word-vectors"
index_opts = dict(settings=dict(number_of_shards=1, number_of_replicas=0))
es.indices.create(index, body=json.dumps(index_opts))

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'elastiknn-example-glove-word-vectors'}

In [31]:
# Define the mapping. Using Permutation LSH model for the vector and keyword for the word.
word_field = "word"
vec_field = "vec"

mapping = {
    "properties": {
        word_field: {
            "type": "keyword"
        },
        vec_field: {
            "type": "elastiknn_dense_float_vector",
            "elastiknn": {
                "dims": 50,
                "model": "permutation_lsh",
                "similarity": "angular",
                "k": 20,
                "repeating": False   
            }
        }
    }
}

es.indices.put_mapping(index=index, body=json.dumps(mapping))

{'acknowledged': True}

In [ ]:
# Index the words and their corresponding vectors using the 